# 11_expert_topics
Performance tuning, hybrid pipelines

In [ ]:
% Content to be added

# File: notebooks/11_expert_topics.ipynb

# OctaveMasterPro: Expert Topics

Master advanced computational techniques and hybrid pipelines! This notebook covers performance tuning, memory optimization, algorithm implementation, numerical methods, and integration with external tools for expert-level scientific computing.

**Learning Objectives:**
- Implement advanced performance optimization techniques
- Master memory management and efficient algorithms
- Create hybrid computational pipelines
- Integrate Octave with external tools and languages
- Apply expert-level numerical methods and custom algorithms

---

## 1. Advanced Performance Tuning

```octave
% Advanced performance optimization techniques
fprintf('=== Advanced Performance Tuning ===\n');

% Memory-efficient algorithms
fprintf('1. Memory-Efficient Algorithms:\n');

function [result, stats] = efficient_matrix_multiply(A, B, block_size)
    % Block-wise matrix multiplication for memory efficiency
    % Input: A, B - matrices to multiply, block_size - block size for tiling
    % Output: result - A*B, stats - performance statistics
    
    if nargin < 3, block_size = 64; end
    
    [m, k1] = size(A);
    [k2, n] = size(B);
    
    if k1 ~= k2
        error('Matrix dimensions incompatible');
    end
    k = k1;
    
    stats = struct();
    stats.original_memory = (m*k + k*n + m*n) * 8;  % Bytes (assuming double)
    stats.block_size = block_size;
    
    tic;
    result = zeros(m, n);
    
    % Block-wise multiplication
    for i = 1:block_size:m
        i_end = min(i + block_size - 1, m);
        for j = 1:block_size:n
            j_end = min(j + block_size - 1, n);
            for kk = 1:block_size:k
                kk_end = min(kk + block_size - 1, k);
                
                # Accumulate block products
                result(i:i_end, j:j_end) = result(i:i_end, j:j_end) + ...
                    A(i:i_end, kk:kk_end) * B(kk:kk_end, j:j_end);
            end
        end
    end
    
    stats.computation_time = toc;
    stats.blocks_processed = ceil(m/block_size) * ceil(n/block_size) * ceil(k/block_size);
    
    fprintf('   Block matrix multiply: %dx%d * %dx%d\n', m, k, k, n);
    fprintf('   Block size: %d, Blocks processed: %d\n', block_size, stats.blocks_processed);
    fprintf('   Time: %.4f seconds\n', stats.computation_time);
end

% Test memory-efficient multiplication
A_test = rand(200, 150);
B_test = rand(150, 180);

[result_blocked, stats_blocked] = efficient_matrix_multiply(A_test, B_test, 50);

% Compare with built-in
tic;
result_builtin = A_test * B_test;
builtin_time = toc;

max_diff = max(abs(result_blocked(:) - result_builtin(:)));
fprintf('   Built-in time: %.4f seconds\n', builtin_time);
fprintf('   Max difference: %.2e\n', max_diff);

% Cache-aware algorithms
fprintf('\n2. Cache-Aware Algorithms:\n');

function [sorted_data, stats] = cache_aware_sort(data, cache_size)
    % Cache-aware merge sort implementation
    % Input: data - array to sort, cache_size - simulated cache size
    % Output: sorted_data - sorted array, stats - performance metrics
    
    if nargin < 2, cache_size = 1024; end
    
    n = length(data);
    stats = struct();
    stats.cache_size = cache_size;
    stats.cache_misses = 0;
    stats.comparisons = 0;
    
    tic;
    if n <= cache_size
        # Use simple sort for small arrays (fits in cache)
        sorted_data = sort(data);
        stats.method = 'in_cache_sort';
    else
        # Use cache-aware merge sort
        sorted_data = cache_merge_sort(data, cache_size, stats);
        stats.method = 'cache_aware_merge';
    end
    stats.time = toc;
    
    fprintf('   Cache-aware sort: %d elements\n', n);
    fprintf('   Method: %s, Time: %.4f seconds\n', stats.method, stats.time);
end

function sorted_data = cache_merge_sort(data, cache_size, stats)
    n = length(data);
    
    if n <= cache_size
        sorted_data = sort(data);
        return;
    end
    
    # Divide
    mid = floor(n/2);
    left = cache_merge_sort(data(1:mid), cache_size, stats);
    right = cache_merge_sort(data(mid+1:end), cache_size, stats);
    
    # Merge
    sorted_data = cache_merge(left, right, stats);
end

function merged = cache_merge(left, right, stats)
    merged = zeros(1, length(left) + length(right));
    i = 1; j = 1; k = 1;
    
    while i <= length(left) && j <= length(right)
        stats.comparisons = stats.comparisons + 1;
        if left(i) <= right(j)
            merged(k) = left(i);
            i = i + 1;
        else
            merged(k) = right(j);
            j = j + 1;
        end
        k = k + 1;
    end
    
    # Copy remaining elements
    while i <= length(left)
        merged(k) = left(i);
        i = i + 1;
        k = k + 1;
    end
    
    while j <= length(right)
        merged(k) = right(j);
        j = j + 1;
        k = k + 1;
    end
end

% Test cache-aware sorting
test_data = rand(1, 5000);
[sorted_cache, cache_stats] = cache_aware_sort(test_data, 512);

fprintf('   Comparisons: %d\n', cache_stats.comparisons);

% Verify correctness
builtin_sorted = sort(test_data);
sort_error = max(abs(sorted_cache - builtin_sorted));
fprintf('   Sorting error: %.2e\n', sort_error);

% Advanced vectorization
fprintf('\n3. Advanced Vectorization Techniques:\n');

function [result, speedup] = vectorized_operations_demo(n)
    % Demonstrate advanced vectorization techniques
    % Input: n - problem size
    % Output: result - computation result, speedup - vectorization speedup
    
    fprintf('   Advanced vectorization demo (n = %d):\n', n);
    
    # Generate test data
    A = rand(n, n);
    B = rand(n, n);
    x = rand(n, 1);
    
    % Method 1: Nested loops (naive)
    tic;
    result_loop = zeros(n, 1);
    for i = 1:n
        for j = 1:n
            result_loop(i) = result_loop(i) + A(i,j) * B(j,i) * x(j);
        end
    end
    loop_time = toc;
    
    % Method 2: Partial vectorization
    tic;
    result_partial = zeros(n, 1);
    for i = 1:n
        result_partial(i) = sum(A(i,:) .* B(:,i)' .* x');
    end
    partial_time = toc;
    
    # Method 3: Full vectorization
    tic;
    result_full = sum((A .* B') .* repmat(x', n, 1), 2);
    full_time = toc;
    
    % Method 4: Optimized vectorization using broadcasting
    tic;
    result_optimal = sum(A .* B' .* x', 2);
    optimal_time = toc;
    
    fprintf('     Loop time: %.6f seconds\n', loop_time);
    fprintf('     Partial vectorization: %.6f seconds (%.1fx speedup)\n', ...
            partial_time, loop_time/partial_time);
    fprintf('     Full vectorization: %.6f seconds (%.1fx speedup)\n', ...
            full_time, loop_time/full_time);
    fprintf('     Optimal vectorization: %.6f seconds (%.1fx speedup)\n', ...
            optimal_time, loop_time/optimal_time);
    
    # Verify results
    max_diff = max([
        max(abs(result_loop - result_partial));
        max(abs(result_loop - result_full));
        max(abs(result_loop - result_optimal))
    ]);
    fprintf('     Max difference between methods: %.2e\n', max_diff);
    
    result = result_optimal;
    speedup = loop_time / optimal_time;
end

[vec_result, vec_speedup] = vectorized_operations_demo(500);
```

## 2. Custom Algorithm Implementation

```octave
% Custom high-performance algorithms
fprintf('\n=== Custom Algorithm Implementation ===\n');

% Fast Fourier Transform implementation
fprintf('1. Custom FFT Implementation:\n');

function X = custom_fft(x)
    % Custom FFT implementation using Cooley-Tukey algorithm
    % Input: x - input signal (length must be power of 2)
    % Output: X - FFT of input signal
    
    N = length(x);
    
    # Base case
    if N == 1
        X = x;
        return;
    end
    
    if mod(N, 2) ~= 0
        error('FFT length must be power of 2');
    end
    
    # Divide
    x_even = x(1:2:end);
    x_odd = x(2:2:end);
    
    # Conquer
    X_even = custom_fft(x_even);
    X_odd = custom_fft(x_odd);
    
    # Combine
    X = zeros(N, 1);
    for k = 1:N/2
        t = exp(-2i * pi * (k-1) / N) * X_odd(k);
        X(k) = X_even(k) + t;
        X(k + N/2) = X_even(k) - t;
    end
end

% Test custom FFT
n_fft = 256;
test_signal = sin(2*pi*(1:n_fft)/n_fft) + 0.5*cos(2*pi*5*(1:n_fft)/n_fft);

tic;
custom_result = custom_fft(test_signal');
custom_fft_time = toc;

tic;
builtin_result = fft(test_signal);
builtin_fft_time = toc;

fft_error = max(abs(custom_result - builtin_result'));
fprintf('   FFT comparison (n = %d):\n', n_fft);
fprintf('   Custom FFT time: %.6f seconds\n', custom_fft_time);
fprintf('   Built-in FFT time: %.6f seconds\n', builtin_fft_time);
fprintf('   Max error: %.2e\n', fft_error);

% Advanced numerical integration
fprintf('\n2. Advanced Numerical Integration:\n');

function [integral, error_est, stats] = adaptive_quadrature(func, a, b, tol, max_depth)
    % Adaptive quadrature using Simpson's rule with error estimation
    % Input: func - function to integrate, [a,b] - interval, tol - tolerance
    % Output: integral - estimated integral, error_est - error estimate
    
    if nargin < 4, tol = 1e-6; end
    if nargin < 5, max_depth = 20; end
    
    stats = struct('function_evals', 0, 'subdivisions', 0, 'max_depth_reached', 0);
    
    [integral, error_est] = adaptive_simpson(func, a, b, tol, max_depth, 0, stats);
    
    fprintf('   Adaptive quadrature results:\n');
    fprintf('   Integral: %.10f\n', integral);
    fprintf('   Error estimate: %.2e\n', error_est);
    fprintf('   Function evaluations: %d\n', stats.function_evals);
    fprintf('   Subdivisions: %d\n', stats.subdivisions);
end

function [integral, error_est] = adaptive_simpson(func, a, b, tol, max_depth, depth, stats)
    if depth > max_depth
        stats.max_depth_reached = stats.max_depth_reached + 1;
        integral = simpson_rule(func, a, b, stats);
        error_est = inf;
        return;
    end
    
    # Simpson's rule on whole interval
    S1 = simpson_rule(func, a, b, stats);
    
    # Simpson's rule on two halves
    c = (a + b) / 2;
    S2 = simpson_rule(func, a, c, stats) + simpson_rule(func, c, b, stats);
    
    # Error estimate (Richardson extrapolation)
    error_est = abs(S2 - S1) / 15;  # For Simpson's rule
    
    if error_est < tol
        integral = S2 + (S2 - S1) / 15;  # Richardson improvement
    else
        stats.subdivisions = stats.subdivisions + 1;
        [I1, E1] = adaptive_simpson(func, a, c, tol/2, max_depth, depth+1, stats);
        [I2, E2] = adaptive_simpson(func, c, b, tol/2, max_depth, depth+1, stats);
        integral = I1 + I2;
        error_est = E1 + E2;
    end
end

function integral = simpson_rule(func, a, b, stats)
    # Simpson's 1/3 rule
    h = (b - a) / 2;
    fa = func(a);
    fc = func((a + b) / 2);
    fb = func(b);
    
    integral = h/3 * (fa + 4*fc + fb);
    stats.function_evals = stats.function_evals + 3;
end

% Test adaptive quadrature
test_func = @(x) exp(-x.^2) .* sin(5*x);  % Oscillatory function
[quad_result, quad_error, quad_stats] = adaptive_quadrature(test_func, 0, 2, 1e-8);

# Compare with built-in integration (if available)
% builtin_result = integral(test_func, 0, 2);
% fprintf('   Built-in integral: %.10f\n', builtin_result);

% High-precision arithmetic simulation
fprintf('\n3. High-Precision Arithmetic:\n');

function result = high_precision_sum(values, method)
    % High-precision summation algorithms
    % Input: values - array to sum, method - 'kahan' or 'pairwise'
    % Output: result - high-precision sum
    
    if nargin < 2, method = 'kahan'; end
    
    switch lower(method)
        case 'kahan'
            result = kahan_sum(values);
        case 'pairwise'
            result = pairwise_sum(values, 1, length(values));
        otherwise
            result = sum(values);
    end
end

function s = kahan_sum(values)
    % Kahan summation algorithm for reduced numerical error
    s = 0;
    c = 0;  % Running compensation for lost low-order bits
    
    for i = 1:length(values)
        y = values(i) - c;    % Recover the low-order part
        t = s + y;            % Add to accumulated sum
        c = (t - s) - y;      % Get the difference (low-order part)
        s = t;                % Update sum
    end
end

function s = pairwise_sum(values, i, j)
    % Pairwise summation for improved precision
    if j - i < 2
        if i == j
            s = values(i);
        else
            s = values(i) + values(j);
        end
    else
        m = floor((i + j) / 2);
        s = pairwise_sum(values, i, m) + pairwise_sum(values, m+1, j);
    end
end

% Test high-precision summation
n_terms = 10000;
test_values = rand(n_terms, 1) * 1e-10;  % Small values to test precision

regular_sum = sum(test_values);
kahan_result = high_precision_sum(test_values, 'kahan');
pairwise_result = high_precision_sum(test_values, 'pairwise');

fprintf('   High-precision summation (%d terms):\n', n_terms);
fprintf('   Regular sum: %.15e\n', regular_sum);
fprintf('   Kahan sum: %.15e\n', kahan_result);
fprintf('   Pairwise sum: %.15e\n', pairwise_result);
fprintf('   Kahan vs regular: %.2e difference\n', abs(kahan_result - regular_sum));
fprintf('   Pairwise vs regular: %.2e difference\n', abs(pairwise_result - regular_sum));
```

## 3. Hybrid Computational Pipelines

```octave
% Hybrid computational pipelines and workflows
fprintf('\n=== Hybrid Computational Pipelines ===\n');

% Pipeline architecture
fprintf('1. Computational Pipeline Framework:\n');

function obj = ComputationalPipeline(name)
    % Framework for building computational pipelines
    obj.name = name;
    obj.stages = {};
    obj.data_flow = struct();
    obj.performance_metrics = struct();
    obj.class_name = 'ComputationalPipeline';
    
    obj.add_stage = @(stage) add_pipeline_stage(obj, stage);
    obj.remove_stage = @(name) remove_pipeline_stage(obj, name);
    obj.execute = @(input_data) execute_pipeline(obj, input_data);
    obj.get_metrics = @() get_pipeline_metrics(obj);
    obj.visualize = @() visualize_pipeline(obj);
end

function add_pipeline_stage(obj, stage)
    obj.stages{end+1} = stage;
    fprintf('   Added pipeline stage: %s\n', stage.name);
end

function remove_pipeline_stage(obj, stage_name)
    for i = length(obj.stages):-1:1
        if strcmp(obj.stages{i}.name, stage_name)
            obj.stages(i) = [];
            fprintf('   Removed pipeline stage: %s\n', stage_name);
            break;
        end
    end
end

function result = execute_pipeline(obj, input_data)
    fprintf('   Executing pipeline: %s\n', obj.name);
    
    current_data = input_data;
    total_time = 0;
    
    for i = 1:length(obj.stages)
        stage = obj.stages{i};
        fprintf('     Stage %d: %s...', i, stage.name);
        
        tic;
        current_data = stage.process(current_data);
        stage_time = toc;
        
        total_time = total_time + stage_time;
        fprintf(' %.4f seconds\n', stage_time);
        
        # Store performance metrics
        obj.performance_metrics.(stage.name) = stage_time;
    end
    
    obj.performance_metrics.total_time = total_time;
    result = current_data;
    
    fprintf('   Pipeline completed in %.4f seconds\n', total_time);
end

function metrics = get_pipeline_metrics(obj)
    metrics = obj.performance_metrics;
    fprintf('   Pipeline performance metrics:\n');
    
    stage_names = fieldnames(metrics);
    for i = 1:length(stage_names)
        name = stage_names{i};
        time = metrics.(name);
        if strcmp(name, 'total_time')
            fprintf('     Total: %.4f seconds\n', time);
        else
            percent = 100 * time / metrics.total_time;
            fprintf('     %s: %.4f seconds (%.1f%%)\n', name, time, percent);
        end
    end
end

function visualize_pipeline(obj)
    fprintf('   Pipeline visualization: %s\n', obj.name);
    fprintf('   ');
    for i = 1:length(obj.stages)
        fprintf('[%s]', obj.stages{i}.name);
        if i < length(obj.stages)
            fprintf(' -> ');
        end
    end
    fprintf('\n');
end

% Pipeline stage templates
function stage = DataPreprocessingStage()
    stage.name = 'DataPreprocessing';
    stage.process = @(data) preprocess_data(data);
end

function processed_data = preprocess_data(data)
    % Simulate data preprocessing
    processed_data = data;
    
    % Remove outliers using IQR method
    if isvector(data)
        Q1 = quantile(data, 0.25);
        Q3 = quantile(data, 0.75);
        IQR = Q3 - Q1;
        lower_bound = Q1 - 1.5 * IQR;
        upper_bound = Q3 + 1.5 * IQR;
        
        outlier_mask = (data < lower_bound) | (data > upper_bound);
        processed_data(outlier_mask) = median(data);
    end
    
    % Normalize data
    processed_data = (processed_data - mean(processed_data)) / std(processed_data);
end

function stage = FeatureExtractionStage()
    stage.name = 'FeatureExtraction';
    stage.process = @(data) extract_features(data);
end

function features = extract_features(data)
    % Extract statistical features from data
    features = struct();
    
    if isvector(data)
        features.mean = mean(data);
        features.std = std(data);
        features.skewness = skewness(data);
        features.kurtosis = kurtosis(data);
        features.min = min(data);
        features.max = max(data);
        features.range = range(data);
        features.energy = sum(data.^2);
        features.zero_crossings = sum(diff(sign(data)) ~= 0);
        
        # Frequency domain features (if signal is long enough)
        if length(data) > 10
            fft_data = abs(fft(data));
            features.spectral_centroid = sum((1:length(fft_data)) .* fft_data') / sum(fft_data);
            features.spectral_energy = sum(fft_data.^2);
        end
    else
        features.data_shape = size(data);
        features.total_elements = numel(data);
        features.matrix_rank = rank(data);
        features.condition_number = cond(data);
    end
end

function stage = AnalysisStage()
    stage.name = 'Analysis';
    stage.process = @(features) analyze_features(features);
end

function results = analyze_features(features)
    % Analyze extracted features
    results = struct();
    
    if isstruct(features)
        field_names = fieldnames(features);
        results.num_features = length(field_names);
        
        % Classify based on statistical properties
        if isfield(features, 'skewness')
            if abs(features.skewness) < 0.5
                results.distribution_type = 'symmetric';
            elseif features.skewness > 0.5
                results.distribution_type = 'right_skewed';
            else
                results.distribution_type = 'left_skewed';
            end
        end
        
        if isfield(features, 'kurtosis')
            if features.kurtosis > 3
                results.tail_type = 'heavy_tailed';
            elseif features.kurtosis < 3
                results.tail_type = 'light_tailed';
            else
                results.tail_type = 'normal_tailed';
            end
        end
        
        results.feature_summary = features;
    else
        results.error = 'Invalid feature format';
    end
end

% Test computational pipeline
fprintf('Testing computational pipeline:\n');

pipeline = ComputationalPipeline('SignalAnalysis');
pipeline.add_stage(DataPreprocessingStage());
pipeline.add_stage(FeatureExtractionStage());
pipeline.add_stage(AnalysisStage());

pipeline.visualize();

# Generate test signal
test_signal = sin(2*pi*5*(1:1000)/1000) + 0.3*randn(1,1000) + [zeros(1,800), 5*ones(1,200)];  % Signal with outliers

% Execute pipeline
pipeline_result = pipeline.execute(test_signal);

fprintf('   Pipeline results:\n');
fprintf('     Distribution type: %s\n', pipeline_result.distribution_type);
fprintf('     Tail type: %s\n', pipeline_result.tail_type);
fprintf('     Number of features: %d\n', pipeline_result.num_features);

pipeline.get_metrics();
```

## 4. Integration and Interfacing

```octave
% Integration with external tools and systems
fprintf('\n=== Integration and Interfacing ===\n');

% File format handlers
fprintf('1. Advanced File Format Handling:\n');

function obj = DataExporter()
    % Multi-format data exporter
    obj.class_name = 'DataExporter';
    obj.supported_formats = {'csv', 'json', 'xml', 'binary', 'hdf5'};
    
    obj.export = @(data, filename, format, options) export_data(data, filename, format, options);
    obj.get_formats = @() obj.supported_formats;
end

function success = export_data(data, filename, format, options)
    if nargin < 4, options = struct(); end
    
    success = false;
    
    try
        switch lower(format)
            case 'csv'
                export_csv(data, filename, options);
            case 'json'
                export_json(data, filename, options);
            case 'xml'
                export_xml(data, filename, options);
            case 'binary'
                export_binary(data, filename, options);
            case 'hdf5'
                fprintf('     HDF5 export would require external library\n');
            otherwise
                error('Unsupported format: %s', format);
        end
        
        success = true;
        fprintf('   Exported data to %s (%s format)\n', filename, format);
        
    catch me
        fprintf('   Export failed: %s\n', me.message);
    end
end

function export_csv(data, filename, options)
    fid = fopen(filename, 'w');
    if fid == -1, error('Cannot create file'); end
    
    if isfield(options, 'header') && options.header
        fprintf(fid, 'Column1,Column2,Column3\n');
    end
    
    if isvector(data)
        for i = 1:length(data)
            fprintf(fid, '%.6f\n', data(i));
        end
    else
        for i = 1:size(data, 1)
            fprintf(fid, '%.6f', data(i, 1));
            for j = 2:size(data, 2)
                fprintf(fid, ',%.6f', data(i, j));
            end
            fprintf(fid, '\n');
        end
    end
    
    fclose(fid);
end

function export_json(data, filename, options)
    fid = fopen(filename, 'w');
    if fid == -1, error('Cannot create file'); end
    
    fprintf(fid, '{\n');
    fprintf(fid, '  "data_type": "%s",\n', class(data));
    fprintf(fid, '  "dimensions": [%s],\n', num2str(size(data)));
    
    if isvector(data)
        fprintf(fid, '  "values": [');
        for i = 1:length(data)
            fprintf(fid, '%.6f', data(i));
            if i < length(data), fprintf(fid, ', '); end
        end
        fprintf(fid, ']\n');
    else
        fprintf(fid, '  "matrix": [\n');
        for i = 1:size(data, 1)
            fprintf(fid, '    [');
            for j = 1:size(data, 2)
                fprintf(fid, '%.6f', data(i, j));
                if j < size(data, 2), fprintf(fid, ', '); end
            end
            fprintf(fid, ']');
            if i < size(data, 1), fprintf(fid, ','); end
            fprintf(fid, '\n');
        end
        fprintf(fid, '  ]\n');
    end
    
    fprintf(fid, '}\n');
    fclose(fid);
end

function export_xml(data, filename, options)
    fid = fopen(filename, 'w');
    if fid == -1, error('Cannot create file'); end
    
    fprintf(fid, '<?xml version="1.0" encoding="UTF-8"?>\n');
    fprintf(fid, '<data>\n');
    fprintf(fid, '  <metadata>\n');
    fprintf(fid, '    <type>%s</type>\n', class(data));
    fprintf(fid, '    <dimensions>%s</dimensions>\n', num2str(size(data)));
    fprintf(fid, '  </metadata>\n');
    
    if isvector(data)
        fprintf(fid, '  <vector>\n');
        for i = 1:length(data)
            fprintf(fid, '    <value index="%d">%.6f</value>\n', i, data(i));
        end
        fprintf(fid, '  </vector>\n');
    else
        fprintf(fid, '  <matrix>\n');
        for i = 1:size(data, 1)
            fprintf(fid, '    <row index="%d">\n', i);
            for j = 1:size(data, 2)
                fprintf(fid, '      <col index="%d">%.6f</col>\n', j, data(i, j));
            end
            fprintf(fid, '    </row>\n');
        end
        fprintf(fid, '  </matrix>\n');
    end
    
    fprintf(fid, '</data>\n');
    fclose(fid);
end

function export_binary(data, filename, options)
    fid = fopen(filename, 'wb');
    if fid == -1, error('Cannot create file'); end
    
    # Write header information
    fwrite(fid, size(data), 'int32');
    fwrite(fid, data, 'double');
    
    fclose(fid);
end

% Test data exporter
exporter = DataExporter();
test_matrix = magic(4);

fprintf('Supported formats: %s\n', strjoin(exporter.get_formats(), ', '));

% Export in different formats
exporter.export(test_matrix, 'test_data.csv', 'csv', struct('header', true));
exporter.export(test_matrix, 'test_data.json', 'json');
exporter.export(test_matrix, 'test_data.xml', 'xml');
exporter.export(test_matrix, 'test_data.bin', 'binary');

% System integration simulation
fprintf('\n2. System Integration Framework:\n');

function obj = SystemInterface(system_type)
    % Generic system interface
    obj.system_type = system_type;
    obj.connected = false;
    obj.class_name = 'SystemInterface';
    
    obj.connect = @() connect_system(obj);
    obj.disconnect = @() disconnect_system(obj);
    obj.send_command = @(cmd) send_system_command(obj, cmd);
    obj.get_status = @() get_system_status(obj);
end

function success = connect_system(obj)
    fprintf('   Connecting to %s system...', obj.system_type);
    pause(0.1);  % Simulate connection time
    obj.connected = true;
    success = true;
    fprintf(' Connected\n');
end

function disconnect_system(obj)
    obj.connected = false;
    fprintf('   Disconnected from %s system\n', obj.system_type);
end

function result = send_system_command(obj, command)
    if ~obj.connected
        error('System not connected');
    end
    
    fprintf('   Sending command to %s: %s\n', obj.system_type, command);
    
    # Simulate different responses based on command
    switch lower(command)
        case 'status'
            result = 'System operational';
        case 'data'
            result = rand(10, 1);  % Simulated data
        case 'compute'
            result = struct('computation_id', randi(1000), 'status', 'submitted');
        otherwise
            result = 'Command executed';
    end
end

function status = get_system_status(obj)
    if obj.connected
        status = struct('connected', true, 'system_type', obj.system_type, ...
                       'last_update', now());
    else
        status = struct('connected', false, 'system_type', obj.system_type);
    end
end

% Test system integration
database_interface = SystemInterface('Database');
compute_interface = SystemInterface('HPC_Cluster');

database_interface.connect();
compute_interface.connect();

db_status = database_interface.send_command('status');
compute_job = compute_interface.send_command('compute');

fprintf('   Database status: %s\n', db_status);
fprintf('   Compute job: ID %d, Status %s\n', compute_job.computation_id, compute_job.status);

database_interface.disconnect();
compute_interface.disconnect();

% Clean up test files
fprintf('\n3. Cleanup:\n');
test_files = {'test_data.csv', 'test_data.json', 'test_data.xml', 'test_data.bin'};
for i = 1:length(test_files)
    if exist(test_files{i}, 'file')
        delete(test_files{i});
        fprintf('   Deleted %s\n', test_files{i});
    end
end
```

## 5. Advanced Numerical Methods

```octave
% Advanced numerical methods and algorithms
fprintf('\n=== Advanced Numerical Methods ===\n');

% Spectral methods
fprintf('1. Spectral Methods:\n');

function [solution, convergence] = spectral_poisson_1d(n, f_func, boundary_conditions)
    % Solve 1D Poisson equation using spectral methods
    % -u''(x) = f(x) on [0,1] with boundary conditions
    % Input: n - number of grid points, f_func - RHS function, boundary_conditions - struct
    % Output: solution - approximate solution, convergence - convergence data
    
    % Chebyshev grid points
    x = cos(pi * (0:n-1) / (n-1));  % Chebyshev points on [-1,1]
    x = (x + 1) / 2;  % Map to [0,1]
    
    % Differentiation matrix (2nd derivative)
    D2 = chebyshev_diff_matrix(n, 2);
    
    % Right-hand side
    f = f_func(x');
    
    % Apply boundary conditions (Dirichlet)
    if isfield(boundary_conditions, 'left') && isfield(boundary_conditions, 'right')
        % Modify system for boundary conditions
        A = -D2(2:n-1, 2:n-1);
        rhs = f(2:n-1) - D2(2:n-1, 1) * boundary_conditions.left - D2(2:n-1, n) * boundary_conditions.right;
        
        % Solve system
        u_interior = A \ rhs;
        
        % Construct full solution
        solution = [boundary_conditions.left; u_interior; boundary_conditions.right];
    else
        error('Dirichlet boundary conditions required');
    end
    
    convergence = struct('grid_points', n, 'method', 'spectral_chebyshev');
    fprintf('   Spectral Poisson solver: %d grid points\n', n);
end

function D = chebyshev_diff_matrix(n, order)
    % Construct Chebyshev differentiation matrix
    if order == 1
        x = cos(pi * (0:n-1) / (n-1));
        c = [2; ones(n-2, 1); 2] .* (-1).^(0:n-1)';
        X = repmat(x', 1, n);
        dX = X - X';
        D = (c * (1./c)') ./ (dX + eye(n));
        D = D - diag(sum(D, 2));
    elseif order == 2
        D1 = chebyshev_diff_matrix(n, 1);
        D = D1 * D1;
    else
        error('Higher order derivatives not implemented');
    end
end

% Test spectral method
test_rhs = @(x) -pi^2 * sin(pi * x);  % RHS for u = sin(pi*x)
bc = struct('left', 0, 'right', 0);  % Homogeneous Dirichlet BCs

[spectral_sol, spec_conv] = spectral_poisson_1d(32, test_rhs, bc);
x_test = cos(pi * (0:31) / 31);
x_test = (x_test + 1) / 2;
exact_sol = sin(pi * x_test');

spectral_error = max(abs(spectral_sol - exact_sol));
fprintf('   Spectral method error: %.2e\n', spectral_error);

% Multigrid method framework
fprintf('\n2. Multigrid Method Framework:\n');

function [solution, iterations] = multigrid_solver(A, b, levels, max_iter, tol)
    % Simplified multigrid solver framework
    % Input: A - system matrix, b - RHS, levels - number of levels
    % Output: solution - approximate solution, iterations
    
    n = length(b);
    solution = zeros(n, 1);
    
    fprintf('   Multigrid solver: %d levels, matrix size %d\n', levels, n);
    
    for iter = 1:max_iter
        old_solution = solution;
        
        % V-cycle
        solution = multigrid_vcycle(A, b, solution, levels);
        
        % Check convergence
        residual_norm = norm(A * solution - b);
        relative_error = norm(solution - old_solution) / norm(solution);
        
        if residual_norm < tol || relative_error < tol
            iterations = iter;
            fprintf('   Converged in %d iterations, residual: %.2e\n', iter, residual_norm);
            return;
        end
        
        if mod(iter, 10) == 0
            fprintf('     Iteration %d: residual %.2e\n', iter, residual_norm);
        end
    end
    
    iterations = max_iter;
    fprintf('   Maximum iterations reached\n');
end

function u = multigrid_vcycle(A, b, u, level)
    if level == 1
        % Coarsest grid - direct solve
        u = A \ b;
    else
        % Pre-smoothing (Gauss-Seidel)
        u = gauss_seidel_smooth(A, b, u, 2);
        
        % Compute residual
        r = b - A * u;
        
        % Restrict to coarser grid (simplified - would use proper restriction)
        n_coarse = floor(length(r) / 2);
        r_coarse = r(1:2:2*n_coarse);
        A_coarse = A(1:2:2*n_coarse, 1:2:2*n_coarse);  % Simplified
        
        % Solve on coarse grid
        e_coarse = multigrid_vcycle(A_coarse, r_coarse, zeros(length(r_coarse), 1), level - 1);
        
        # Prolongate correction (simplified)
        e = zeros(size(u));
        e(1:2:2*length(e_coarse)) = e_coarse;
        
        % Correct solution
        u = u + e;
        
        % Post-smoothing
        u = gauss_seidel_smooth(A, b, u, 2);
    end
end

function u = gauss_seidel_smooth(A, b, u, iterations)
    n = length(u);
    for iter = 1:iterations
        for i = 1:n
            u(i) = (b(i) - A(i, 1:i-1) * u(1:i-1) - A(i, i+1:n) * u(i+1:n)) / A(i, i);
        end
    end
end

% Test multigrid (simplified example)
n_mg = 64;
A_mg = 2 * eye(n_mg) - diag(ones(n_mg-1, 1), 1) - diag(ones(n_mg-1, 1), -1);  % Tridiagonal
b_mg = ones(n_mg, 1);

[mg_solution, mg_iter] = multigrid_solver(A_mg, b_mg, 3, 100, 1e-8);

% Adaptive mesh refinement
fprintf('\n3. Adaptive Mesh Refinement:\n');

function [refined_mesh, error_indicators] = adaptive_refine(mesh, solution, tolerance)
    % Adaptive mesh refinement based on error indicators
    % Input: mesh - current mesh, solution - current solution, tolerance
    % Output: refined_mesh - new mesh, error_indicators - element errors
    
    n_elements = length(mesh) - 1;
    error_indicators = zeros(n_elements, 1);
    
    # Compute error indicators (simplified gradient-based)
    for i = 1:n_elements
        h = mesh(i+1) - mesh(i);
        if i == 1
            grad_approx = abs(solution(i+1) - solution(i)) / h;
        elseif i == n_elements
            grad_approx = abs(solution(i) - solution(i-1)) / h;
        else
            grad_left = abs(solution(i) - solution(i-1)) / (mesh(i) - mesh(i-1));
            grad_right = abs(solution(i+1) - solution(i)) / h;
            grad_approx = max(grad_left, grad_right);
        end
        
        error_indicators(i) = h * grad_approx;
    end
    
    % Refine elements with large error
    refined_mesh = mesh;
    elements_refined = 0;
    
    for i = n_elements:-1:1  % Process backwards to maintain indexing
        if error_indicators(i) > tolerance
            # Insert midpoint
            midpoint = (mesh(i) + mesh(i+1)) / 2;
            refined_mesh = [refined_mesh(1:i), midpoint, refined_mesh(i+1:end)];
            elements_refined = elements_refined + 1;
        end
    end
    
    fprintf('   Adaptive refinement: %d elements refined\n', elements_refined);
    fprintf('   New mesh size: %d points (was %d)\n', length(refined_mesh), length(mesh));
end

% Test adaptive refinement
initial_mesh = linspace(0, 1, 21);
test_solution = exp(-10 * (initial_mesh - 0.7).^2);  % Sharp peak at x=0.7

[refined_mesh, error_est] = adaptive_refine(initial_mesh, test_solution, 0.1);
fprintf('   Max error indicator: %.4f\n', max(error_est));
```

---

# Summary

**Expert Topics Mastery Completed:**

This advanced notebook covered cutting-edge computational techniques and professional-grade implementations:

- ✅ **Performance Tuning**: Memory-efficient algorithms, cache-aware computing, advanced vectorization
- ✅ **Custom Algorithms**: FFT implementation, adaptive quadrature, high-precision arithmetic  
- ✅ **Hybrid Pipelines**: Computational workflow frameworks, multi-stage processing systems
- ✅ **System Integration**: Multi-format data export, external system interfaces, file handling
- ✅ **Advanced Numerics**: Spectral methods, multigrid solvers, adaptive mesh refinement

**Expert-Level Computational Skills:**
1. **Algorithm Design**: Implement complex algorithms with optimal performance characteristics
2. **Memory Management**: Design cache-efficient and memory-conscious computational strategies
3. **Pipeline Architecture**: Build sophisticated multi-stage computational workflows
4. **System Integration**: Interface with external tools, databases, and computing systems
5. **Advanced Numerics**: Apply state-of-the-art numerical methods for complex problems

**Professional Software Development:**
- Design high-performance computational algorithms from scratch
- Implement memory-efficient data structures and processing pipelines
- Create robust interfaces for system integration and data exchange
- Apply advanced numerical methods for specialized scientific computing
- Build scalable frameworks for complex computational workflows

**Research and Industry Applications:**
- **High-Performance Computing**: Large-scale scientific simulations and modeling
- **Computational Science**: Advanced numerical methods for research applications
- **Data Processing**: Industrial-scale data analysis and transformation pipelines
- **System Integration**: Enterprise software connecting multiple computational resources
- **Algorithm Development**: Creating new methods for specialized computational problems

**Best Practices Established:**
- Profile and optimize algorithms systematically for production performance
- Design modular, extensible computational frameworks
- Implement robust error handling and validation in complex systems
- Use appropriate numerical methods based on problem characteristics
- Build comprehensive testing and benchmarking for custom algorithms

**Next Steps:**
- Apply these techniques to domain-specific research or industry problems
- Explore GPU computing and massively parallel algorithms
- Study advanced topics in computational mathematics and computer science
- Proceed to `12_parallel_computing.ipynb` for the final advanced topic

Your expert-level computational skills are now ready for the most challenging scientific and engineering applications! 🚀🔬